# HW03: Supervised Machine Learning, Regression and XGBoost

Remember that these homework work as a completion grade. **You can skip one section without losing credit.**

## Load and Pre-process Text
We do sentiment analysis on the [Movie Review Data](https://www.cs.cornell.edu/people/pabo/movie-review-data/). If you would like to know more about the data, have a look at [the paper](https://www.cs.cornell.edu/home/llee/papers/pang-lee-stars.pdf) (but no need to do so).

In [1]:
# In this tutorial, we do sentiment analysis
# download the data
#!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar xf aclImdb_v1.tar.gz

!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
 
!tar xf scale_data.tar.gz 
!tar xf scale_whole_review.tar.gz

--2022-03-22 17:05:18--  https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3.8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz.1’

scale_data.tar.gz.1 100%[===================>]   3.84M  3.38MB/s    in 1.1s    

2022-03-22 17:05:20 (3.38 MB/s) - ‘scale_data.tar.gz.1’ saved [4029756/4029756]

--2022-03-22 17:05:20--  https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8853204 (8.4M) [application/x-gzip]
Saving to: ‘scale_whole_review.tar.gz.1’

scale_whole_review. 100%[===================>]   8.44M  5.57

First, we have to load the data for which we provide the function below. Note how we also preprocess the text using gensim's simple_preprocess() function and how we already split the data into a train and test split.

In [2]:
import os
from gensim.utils import simple_preprocess
def load_data():
    examples, labels = [], []
    authors = os.listdir("scale_whole_review")
    for author in authors:
        path = os.listdir(os.path.join("scale_whole_review", author, "txt.parag"))
        fn_ids = os.path.join("scaledata", author, "id." + author)
        fn_ratings = os.path.join("scaledata", author, "rating." + author)
        with open(fn_ids) as ids, open(fn_ratings) as ratings:
            for idx, rating in zip(ids, ratings):
                labels.append(float(rating.strip()))
                filename_text = os.path.join("scale_whole_review", author, "txt.parag", idx.strip() + ".txt")
                with open(filename_text, encoding='latin-1') as f:
                    examples.append(" ".join(simple_preprocess(f.read())))
    return examples, labels
                  
X,y  = load_data()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print ("text:", X_train[0], "\nlabel:", y_train[0])

text: bloody child the director writer cinematographer nina menkes screenwriter tinka menkes editors nina and tina menkes cast tinka menkes captain sherry sibley murdered wife robert mueller murderer russ little sergeant jack hara enlisted man runtime mirage reviewed by dennis schwartz an amazingly strange film confusing and not thoroughly enjoyable but film found more interesting than thought possible at first viewing this experimental film in minimalist story telling film consisting of disturbing visualizations and almost no dialogue had concept that was greater than how the film turned out it felt at times like was watching paint dry on the wall but the reward for sitting through those excruciatingly redundant scenes was in seeing something different something that cast spell of sorcery over terrible incident as believe the film in its unique and sometimes shrill voice does justice in commenting on the violence in american society especially against women the film uses its impressio

## Vectorize the data

In [3]:
# train a TF_IDF Vectorizer on X_train and vectorize X_train and X_test
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=0.01, # at min 1% of docs
                        max_df=.5,  
                        stop_words='english',
                        ngram_range=(1,2))

# TODO train vectorizer
vec.fit(X_train)

# TODO transform X_train to TF-IDF values
X_train_tfidf = vec.transform(X_train)
# TODO transform X_test to TF-IDF values
X_test_tfidf = vec.transform(X_test)

In [4]:
# TODO scale both training and test data with the standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)

X_train_tfidf_scaled = scaler.fit_transform(X_train_tfidf)
X_test_tfidf_scaled = scaler.transform(X_test_tfidf)

In [5]:
# TODO train an elastic net on the transformed output of the scaler
from sklearn.linear_model import ElasticNet

en = ElasticNet(alpha=0.01)

# TODO train the ElasticNet
en.fit(X_train_tfidf_scaled, y_train)

# TODO predict the testset
y_head = en.predict(X_test_tfidf_scaled)

from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, balanced_accuracy_score
# TODO print mean squared error and r2 score on the test set

print('r2_score: ', r2_score(y_true=y_test, y_pred=y_head))
print('mean_squared_error: ', mean_squared_error(y_true=y_test, y_pred=y_head))

r2_score:  0.4831101365982974
mean_squared_error:  0.01635537701694255


Next, we train an OLS model doing binary prediction on these movie reviews. Two get two bins, we transform the continuous ratings into two classes, where one class contains all the negative ratings (value < 0.5), the other class all the positive ratings (value > 0.5)

In [6]:
y_train = [1 if i >= 0.5 else 0 for i in y_train]
y_test = [1 if i >= 0.5 else 0 for i in y_test]


In [10]:
##TODO train logistic regression on X_train
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(penalty='elasticnet', l1_ratio=0.2, solver='saga')

# TODO train a logistic regression
logistic_regression.fit(X_train_tfidf_scaled, y_train)

# TODO predict the testset
y_test_head = logistic_regression.predict(X_test_tfidf_scaled)

# since we have continuous output, we need to post-process our labels into two classes. We choose a threshold of 0.5
def map_predictions(predicted):
    predicted = [1 if i > 0.5 else 0 for i in predicted]
    return predicted

# TODO print the accuracy of our classifier on the testset
from sklearn.metrics import accuracy_score
accuracy_score(y_test, map_predictions(y_test_head))

/home/pucyril/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.8165859564164649

In [54]:
import numpy as np
# TODO print the 10 most informative words of the regression (the 10 words having the highest coefficients)
log_reg_coef = logistic_regression.coef_

N = 10
ind = np.argpartition(log_reg_coef, -N)[0,-N:]
print('weighs: ', log_reg_coef[0,ind])

# invert vocabulary dictionary
inv_vocab = {v: k for k, v in vec.vocabulary_.items()}

print('\nMost informative words:')
for i in ind:
    print('\t', inv_vocab[i])

weighs:  [0.12171678 0.12209349 0.12521556 0.15105691 0.13879401 0.12498745
 0.13114787 0.13059017 0.12200828 0.13739672]

Most informative words:
	 quiet
	 adds
	 question
	 surprisingly
	 chan
	 punches
	 fine
	 true
	 wonderful
	 effective


## XGBoost

Lastly, we train an XGBoost classifier to do topic prediction on the AG news dataset, which is a multi-class prediction problem (4 classes). We again have to vectorize the data, train the classifier, predict the testset and output an evaluation metric (we go for accuracy).

In [55]:
!pip install xgboost

     |████████████████████████████████| 173.6 MB 77.2 MB/s eta 0:00:01     |███████████████████████████████▋| 171.4 MB 77.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/pucyril/bin/python -m pip install --upgrade pip' command.


In [56]:
#Import the AG news dataset (same as hw01)
#Download them from here 
#!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
df["text"] = df["title"] + " " + df["lead"]
df.head()

,label,title,lead,text
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...,"Stocks End Up, But Near Year Lows (Reuters) Re..."


In [57]:
# vectorize the data
from sklearn.feature_extraction.text import TfidfVectorizer

# only consider 10% of the data
dfs = df.sample(frac=0.1)

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(dfs["text"], dfs["label"], test_size=0.33, random_state=42)

vec = TfidfVectorizer(min_df=5, # at min 1% of docs
                        max_df=.5,  
                        stop_words='english',
                        max_features=2000,
                        ngram_range=(1,2))

# transform into TF-IDF values
X_train_tfidf = vec.fit_transform(X_train).todense()
X_test_tfidf = vec.transform(X_test).todense()


XGBoost provides an interface to SKLearn classifiers, e.g. they implement the same train and predict methods as an SKLearn classifier would. If you are interested in a more detailed overview, have a look at the [official documentation](https://xgboost.readthedocs.io/en/latest/python/index.html).

In [59]:
param_dist = {'objective':'multi:softmax', 'num_class': 5, 'n_estimators':25}

# note how we only have 4 labels, but we need to pass "num_class": 5
# if we pass "num_class": 4, we get the error "label must be in [0, num_class)."
import xgboost as xgb

clf = xgb.XGBModel(**param_dist)

# TODO train the XGBModel
clf.fit(X_train_tfidf, y_train)

[17:34:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBModel(base_score=0.5, booster='gbtree', colsample_bylevel=1,
         colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
         interaction_constraints='', learning_rate=0.300000012,
         max_delta_step=0, max_depth=6, min_child_weight=1,
         monotone_constraints='()', n_estimators=25, n_jobs=16, num_class=5,
         num_parallel_tree=1, objective='multi:softmax', predictor='auto',
         random_state=0, reg_alpha=0, reg_lambda=1, subsample=1,
         tree_method='exact', validate_parameters=1)

In [63]:
# TODO predict the testset
y_head = clf.predict(X_test_tfidf)

# TODO evaluate the predictions using accuracy as a metric
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_head)

0.8068181818181818